In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Force TensorFlow to use GPU if available
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    tf.config.set_visible_devices(physical_devices[0], 'GPU')
    print("Using GPU for training.")
else:
    print("Using CPU for training.")

# Load the pre-trained VGG16 model without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Fine-tune the deeper layers of the base model
for layer in base_model.layers[:15]:
    layer.trainable = False

# Dynamically update the output layer to match the number of classes
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
output_layer = Dense(8, activation='softmax')(x)  # Update this to the correct number of classes

# Create and compile the final model
model = Model(inputs=base_model.input, outputs=output_layer)
optimizer = Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Image data generators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    horizontal_flip=True,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

# Define the directory path for train and validation sets
train_data_dir = r"F:\data_image\preprocessed_images"

# Paths for train and validation folders
train_dir = os.path.join(train_data_dir, "train")
val_dir = os.path.join(train_data_dir, "validation")

# Ensure the directories exist, create them if missing
if not os.path.exists(train_dir):
    print(f"Creating missing directory: {train_dir}")
    os.makedirs(train_dir)

if not os.path.exists(val_dir):
    print(f"Creating missing directory: {val_dir}")
    os.makedirs(val_dir)

# Check for class consistency
train_classes = set(os.listdir(train_dir))
val_classes = set(os.listdir(val_dir))

# Identify mismatched classes
missing_in_val = train_classes - val_classes
missing_in_train = val_classes - train_classes

# Fix folder structure by creating missing class directories
for class_name in missing_in_val:
    os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)
for class_name in missing_in_train:
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)

print("Folder structure aligned. Missing folders added where necessary.")

# Prepare the data generators
train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    directory=val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Compute class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = {i: weight for i, weight in enumerate(class_weights)}

print("Class weights computed:", class_weights)

# Callbacks for improved training performance
early_stopping = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, min_lr=1e-6, verbose=1)

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    class_weight=class_weights,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

# Evaluate the model
val_generator.reset()
y_true = val_generator.classes
y_pred = model.predict(val_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

# Metrics
accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

print("Evaluation Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes))

Using GPU for training.
Found 3505 images belonging to 5 classes.
Found 874 images belonging to 5 classes.
Found 1735 images belonging to 5 classes.
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


220/220 ━━━━━━━━━━━━━━━━━━━━ 1324s 6s/step - accuracy: 0.2182 - loss: 1.9892 - val_accuracy: 0.3169 - val_loss: 1.4735 - learning_rate: 1.0000e-04
Epoch 2/50
220/220 ━━━━━━━━━━━━━━━━━━━━ 87s 296ms/step - accuracy: 0.2372 - loss: 1.6623 - val_accuracy: 0.1808 - val_loss: 1.5992 - learning_rate: 1.0000e-04
Epoch 3/50
220/220 ━━━━━━━━━━━━━━━━━━━━ 66s 289ms/step - accuracy: 0.2047 - loss: 1.5200 - val_accuracy: 0.1396 - val_loss: 1.6609 - learning_rate: 1.0000e-04
Epoch 4/50
220/220 ━━━━━━━━━━━━━━━━━━━━ 65s 289ms/step - accuracy: 0.1128 - loss: 1.6591 - val_accuracy: 0.2002 - val_loss: 1.5337 - learning_rate: 1.0000e-04
Epoch 5/50
220/220 ━━━━━━━━━━━━━━━━━━━━ 67s 291ms/step - accuracy: 0.1959 - loss: 1.5674 - val_accuracy: 0.2414 - val_loss: 1.5452 - learning_rate: 1.0000e-04
Epoch 6/50
220/220 ━━━━━━━━━━━━━━━━━━━━ 66s 292ms/step - accuracy: 0.1994 - loss: 1.4510 - val_accuracy: 0.1636 - val_loss: 1.5958 - learning_rate: 1.0000e-04
Epoch 7/50
220/220 ━━━━━━━━━━━━━━━━━━━━ 83s 297ms/step - a

In [ ]:
from google.colab import drive
drive.mount('/content/drive')